<a href="https://www.kaggle.com/code/galenchen/taiwan-highway-vd-project?scriptVersionId=125989129" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Download multiple DATE x VDID loop
## Set parameters.

In [11]:
#***********************************************************************
clear_working_folder = True # so you will only be mailed with the latest results, but make sure the files have been saved.
use_all_subfolders = True # if this is true, then you can only use Kaggle dataset.
realtime = False #will be set to false if you use all subfolders

dataset_list = ['vd-2023-0326-0401'] # If the length of this list is longer than one, use_all_subfolders == True, realtime == False.
date_list = ['20230401']#[yesterday_str] if you want to run data from yesterday
vdid_list = ["VD-N1-S-369.007-M-Loop"]#, "VD-N1-S-369.400-M-Loop", "VD-N1-S-370.000-M-Loop", "VD-N1-S-371.010-M-Loop"]
start_time = "1112"
end_time = "1141"

your_email = 'tudo11927.y@nycu.edu.tw'
#***********************************************************************

## Load the dataset and the packages
* Suggest uploading the xml.gz files onto Kaggle so it doesn't fuck up the output storage.
* Also, the package xmltodict have to be installed before anything happens.

In [4]:
!pip install xmltodict # Install xmltodict package if not already installed

import xmltodict # Import xmltodict module for parsing XML data
import numpy as np # Import numpy and pandas for data analysis and processing
import pandas as pd
import json # Import json module for working with JSON data
from tqdm import tqdm # Import tqdm module for progress bars
import urllib.request # Import urllib.request module for working with URLs
import gzip # Import gzip module for working with compressed data files
from datetime import datetime, timedelta 
import pytz # Import datetime and pytz modules for working with dates and times
import os # Import os module for working with the file system

# Walk through the /kaggle/input directory and print out the path of each file in it
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0035.xml/VDLive_0035.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0523.xml/VDLive_0523.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0204.xml/VDLive_0204.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0730.xml/VDLive_0730.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_2039.xml/VDLive_2039.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_1845.xml/VDLive_1845.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0503.xml/VDLive_0503.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_0113.xml/VDLive_0113.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_1242.xml/VDLive_1242.xml
/kaggle/input/vd-2023-0326-0401/20230401/kaggle/working/20230401/VDLive_2232.xml/VDLive_2232.xml
/kaggle/input/vd-2023-0326-040

## Realtime downloading and conversion pipeline

Download xml.gz file, decompress into xml file, then delete the xml.gz, then feed the xml into get_vd, then delete the xml after data is analyzed.

In [5]:
def download_xml(date, current_time):
    
    global file_exist
    global empty_file
    
    file_exist = False
    empty_file = False

    url = f'https://tisvcloud.freeway.gov.tw/history/motc20/VD/{date}/VDLive_{current_time}.xml.gz'
    filename = f'VDLive_{current_time}.xml.gz'

    # Download the file
    if os.path.isfile(filename):
        if os.path.getsize(filename) > 0 and os.path.getsize(filename) < 1024:
            os.remove(filename)
            empty_file = True
            file_exist = True
        elif os.path.getsize(filename) == 0:
            os.remove(filename)
            file_exist = False
        else: file_exist = True
    else:
        urllib.request.urlretrieve(url, filename)
        if urllib.request.urlopen(url).getcode() == 404:
            file_exist = False
            os.remove(filename)
            return
        elif os.path.getsize(filename) > 0 and os.path.getsize(filename) < 1024:
            os.remove(filename)
            empty_file = True
            file_exist = True
            return
        elif os.path.getsize(filename) == 0:
            os.remove(filename)
            file_exist = False
            return
        else: file_exist = True

    if file_exist and not empty_file:
        # File decompression
        if os.path.isfile(filename):
            with gzip.open(filename, 'rb') as f:
                xml_data = f.read()

            xml_file_path = f'VDLive_{current_time}.xml'

            with open(xml_file_path, 'wb') as f:
                f.write(xml_data)

            if os.path.exists(xml_file_path):
                os.remove(filename)
            else:
                print(f'Error: {xml_file_path} was not created.')
        else:
            print(f"Error: {filename} not found.")

## XML to JSON to DataFrame with pivoting to CSV pipeline

In [6]:
def get_vd(date, time, desired_vdid, realtime, dataset):

    global vdid_found
    global vdid_absent
    vdid_found = False
    

    if realtime:
        directory = f"VDLive_{time}.xml"
    else:
        global file_exist
        global empty_file
        global file_absent_in_dataset
        empty_file = False
        file_exist = False
        directory = f"/kaggle/input/{dataset}/{date}/kaggle/working/{date}/VDLive_{time}.xml/VDLive_{time}.xml"
        if not os.path.isfile(directory):
            file_absent_in_dataset.append(time)
            return

        elif os.path.getsize(directory) > 0 and os.path.getsize(directory) < 1024:
            empty_file = True
            file_exist = True
            return
        elif os.path.getsize(directory) == 0:
            file_exist = False
            return
        else:
            file_exist = True
    #print(file_exist, empty_file)
    
    if file_exist and not empty_file:
        with open(directory) as xml_file:   # Parse XML data into a dictionary
            data_dict = xmltodict.parse(xml_file.read())
        json_data = json.dumps(data_dict) # Convert dictionary to JSON
        #print("XML file converted to JSON.")    
        data = json.loads(json_data) # Load the JSON data into a dictionary
        #print(data)
        # Get the desired VDID, and iterate through the VDLives and find the one with the desired VDID
        for vdlive in data["VDLiveList"]["VDLives"]["VDLive"]:
            if vdlive["VDID"] == desired_vdid:
                # Extract the data for the desired VDID
                link_flows = vdlive["LinkFlows"]["LinkFlow"]
                status = vdlive["Status"]
                date_time = f'{date}_{time}'
                # Print the VDID and its corresponding data
                data= {"VDID": desired_vdid, "LinkFlows": link_flows, "Status": status,
                       #"DateTime": date_time
                      }
                #print(f"Detector {desired_vdid} found.")
                #print(data)
                vdid_found = True
                break

        if vdid_found:
            # Extract data from JSON and flatten into a list of rows
            rows = []
            vdid = data['VDID']
            date_time = f'{date}_{time}'
            link_id = data['LinkFlows']['LinkID']

            # Situation 1: 'Lane' key is present in the 'Lanes' dictionary
            if 'Lane' in data['LinkFlows']['Lanes']:
                lanes = data['LinkFlows']['Lanes']['Lane']
                if isinstance(lanes, dict):
                    # If 'Lane' is a dictionary, convert it to a list with a single element
                    lanes = [lanes]
                for lane in lanes:
                    lane_id = lane['LaneID']
                    lane_type = lane['LaneType']
                    lane_speed = lane['Speed']
                    occupancy = lane['Occupancy']
                    for vehicle in lane['Vehicles']['Vehicle']:
                        vehicle_type = vehicle['VehicleType']
                        volume = vehicle['Volume']
                        speed = vehicle['Speed']
                        rows.append([date_time, vdid, link_id, lane_id,
                                     lane_type, lane_speed, occupancy, 
                                     vehicle_type, volume, speed])

            # Situation 2: 'Lane' key is not present in the 'Lanes' dictionary
            elif 'Lane' in data['LinkFlows']['Lanes']['Lane']:
                lane = data['LinkFlows']['Lanes']['Lane']
                lane_id = lane['LaneID']
                lane_type = lane['LaneType']
                lane_speed = lane['Speed']
                occupancy = lane['Occupancy']
                for vehicle in lane['Vehicles']['Vehicle']:
                    vehicle_type = vehicle['VehicleType']
                    volume = vehicle['Volume']
                    speed = vehicle['Speed']
                    rows.append([date_time, vdid, link_id, lane_id,
                                 lane_type, lane_speed, occupancy,
                                 vehicle_type, volume, speed])
            return rows
        else:
            vdid_absent.append(str(time))


def get_vds(date, desired_vdid, start_time, end_time, realtime, dataset):
    
    global email_text

    # Convert start_time and end_time to integers
    start_time2 = int(start_time)
    str_end_time = end_time
    end_time = int(end_time)

    # Convert start_time and end_time to minutes
    start_time_minutes = (start_time2 // 100) * 60 + (start_time2 % 100)
    end_time_minutes = (end_time // 100) * 60 + (end_time % 100)
    total = end_time_minutes - start_time_minutes + 1

    # Create an empty list to store the results
    results = []
    missing = []
    empty_files =[]
    file_absent_in_dataset =[]
    
    i = 0
    current_time = start_time

    # Loop through the function multiple times and append the result to the list
    with tqdm(total=total) as pbar:
        while i < total:
            
            if realtime == True:
                download_xml(date, current_time)
            result = get_vd(date, current_time, desired_vdid, realtime, dataset) ###
            if vdid_found == True:
                results.extend(result)
            filename = f"/kaggle/working/VDLive_{current_time}.xml"
            if os.path.exists(filename):
                os.remove(filename)               
            if empty_file:
                empty_files.append(str(current_time))
            if not file_exist:
                missing.append(str(current_time))
                
            pbar.update(1)
            i += 1

            if int(current_time) % 100 == 59:
                current_time = str(int(current_time)+41)
            else:
                current_time = str(int(current_time)+1)
            
            if int(current_time)<10:
                current_time = f"000{current_time}"
            elif int(current_time)<100:
                current_time = f"00{current_time}"
            elif int(current_time)<1000:
                current_time = f"0{current_time}"
            else:
                current_time = current_time

    #print(results)

    # Create a pandas DataFrame from the rows list
    df = pd.DataFrame(results, columns=['DateTime', 'VDID', 'LinkID',
                                        'LaneID', 'LaneType',
                                        'LaneSpeed', 'Occupancy',
                                        'VehicleType', 'Volume', 'Speed'])
    #pivot dataframe according to vehicle
    df = df.pivot(index=['DateTime', 'VDID', 'LinkID', 'LaneID',
                     'LaneType', 'LaneSpeed', 'Occupancy'],
              columns='VehicleType',
              values=['Volume', 'Speed'])
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df = df.reset_index()
    df = df.reindex(columns=['DateTime', 'VDID', #'LinkID', 
                             'LaneID',#'LaneType',
                             'LaneSpeed', 'Occupancy',
                             'Volume_S', 'Speed_S', 'Volume_L', 'Speed_L', 'Volume_T', 'Speed_T'])
    
    #pivot the dataframe according to lane
    df = df.pivot_table(index=['DateTime', 'VDID'], columns='LaneID', values=['LaneSpeed', 'Occupancy', 'Volume_S', 'Speed_S', 'Volume_L', 'Speed_L', 'Volume_T', 'Speed_T']).reset_index()
    #flatten the column names
    df.columns = ['_'.join(str(col).strip() for col in tup) for tup in df.columns.values]

    #Save to CSV
    filename = f'{date}_{desired_vdid}_{start_time}_{str_end_time}'
    df.to_csv(f'{filename}.csv', index=False)
    
    
    report_string = (f"{filename}.csv has been saved in /kaggle/working.\n"
      f"missing files = {missing}\n"
      f"files not found in dataset = {file_absent_in_dataset}\n"
      f"empty files = {empty_files}\n"
      f"VDID not found in files = {vdid_absent}\n"
      f"\n")
    
    # Append the report string to the email text using a list
    email_text += report_string
    
    print(report_string)
    #display(df)

## Acquire time so the script does automatic daily updates for yesterday.

In [7]:
# Set the time zone to GMT+8
tz = pytz.timezone('Asia/Taipei')
# Get the current time in GMT+8 time zone
now = datetime.now(tz)
# Subtract one day to get yesterday's date
yesterday = now - timedelta(days=1)
# Format yesterday's date as YYYYMMDD
yesterday_str = yesterday.strftime('%Y%m%d')

print(yesterday_str)


20230415


# Mail execution results and data to the user with Gmail API.

In [8]:
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

def email_report(your_email):

    ## Gmail account credentials
    gmail_user = 'galen147258369@gmail.com'
    gmail_password = 'ikikvtbtlecywjtt'

    # Recipient email address
    to = your_email

    # Create a multipart message container
    msg = MIMEMultipart()

    # Set the subject of the email
    msg['Subject'] = 'Taiwan Highway VD project'

    # Set the sender and recipient of the email
    msg['From'] = gmail_user
    msg['To'] = to

    # Create a text message part
    text = MIMEText(email_text)
    msg.attach(text)

    # Attach all files in the working directory to the email
    working_dir = '/kaggle/working/'
    file_list = [f for f in os.listdir(working_dir) if not f.startswith('.virtual_documents') and os.path.isfile(os.path.join(working_dir, f))]
    for filename in file_list:
        file_path = os.path.join(working_dir, filename)
        with open(file_path, 'rb') as f:
            file_data = f.read()
        attachment = MIMEApplication(file_data, _subtype='csv')
        attachment.add_header('content-disposition', 'attachment', filename=filename)
        msg.attach(attachment)

    # Connect to Gmail's SMTP server and send the email
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login(gmail_user, gmail_password)
    server.sendmail(msg['From'], msg['To'], msg.as_string())
    server.quit()


# RUN THE CODE HERE !

In [12]:
if clear_working_folder == True:
    !rm -rf /kaggle/working/*
    
if len(dataset_list) > 1:
    use_all_subfolders == True
    realtime == False

file_exist = False
empty_file = False
vdid_found = False
file_absent_in_dataset = []
vdid_absent = []
email_text = ""
task = 0

report_string = "Dataset list: " + str(dataset_list) + "\n"
report_string += "Use all subfolders: " + str(use_all_subfolders) + "\n"
report_string += "Date list: " + str(date_list) + "\n"
report_string += "VDID list: " + str(vdid_list) + "\n"
report_string += "Start time: " + start_time + "\n"
report_string += "End time: " + end_time + "\n"
report_string += "Realtime: " + str(realtime) + "\n"
report_string += "Your email: " + your_email + "\n"
report_string += "\n"

email_text += report_string

total_tasks = 0

if use_all_subfolders:
    subfolders = []
    for dataset in dataset_list:
        for folder_name in os.listdir(f'/kaggle/input/{dataset}'):
            if os.path.isdir(os.path.join(f'/kaggle/input/{dataset}', folder_name)):
                subfolders.append(folder_name)
        date_list = subfolders
        total_tasks += len(date_list)*len(vdid_list)
else:
    total_tasks = len(date_list)*len(vdid_list)
    
print(f' Total tasks: {total_tasks}')
print()

task = 0

if use_all_subfolders:
    realtime = False # if use all subfolders, then realtime must be false.
    for dataset in dataset_list:
        subfolders = []
        for folder_name in os.listdir(f'/kaggle/input/{dataset}'):
            if os.path.isdir(os.path.join(f'/kaggle/input/{dataset}', folder_name)):
                subfolders.append(folder_name)
        date_list = subfolders
        for date in date_list:
            for vdid in vdid_list:
                get_vds(date, vdid, start_time, end_time, realtime, dataset)
                task += 1
else:
    for date in date_list:
        for vdid in vdid_list:
            get_vds(date, vdid, start_time, end_time, realtime, dataset_list[0])
            task += 1

loop_complete = f"{task} out of {total_tasks} tasks in the loop were completed."

email_text +=loop_complete
email_text += "\n"
email_text += "\n"

# Get a list of all files in the /kaggle/working/ directory excluding '.virtual_documents' directory
file_list = [f for f in os.listdir('/kaggle/working/') if not f.startswith('.virtual_documents') and os.path.isfile(os.path.join('/kaggle/working/', f))]
# Add the list of files to email_text
email_text += "\n".join([f"Attachment: {filename}" for filename in file_list]) + "\n"

email_text += "\n"
email_text += "END OF MESSAGE"

email_report(your_email)
print("Email sent.")

 Total tasks: 7



100%|██████████| 30/30 [00:11<00:00,  2.64it/s]


20230401_VD-N1-S-369.007-M-Loop_1112_1141.csv has been saved in /kaggle/working.
missing files = []
files not found in dataset = []
empty files = ['1116', '1117', '1118', '1119', '1120', '1121', '1122', '1123', '1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133', '1134', '1135', '1136', '1137', '1138']
VDID not found in files = []




100%|██████████| 30/30 [00:47<00:00,  1.60s/it]


20230330_VD-N1-S-369.007-M-Loop_1112_1141.csv has been saved in /kaggle/working.
missing files = []
files not found in dataset = []
empty files = []
VDID not found in files = []




...

20230326_VD-N1-S-369.007-M-Loop_1112_1141.csv has been saved in /kaggle/working.
missing files = []
files not found in dataset = []
empty files = []
VDID not found in files = []




100%|██████████| 30/30 [00:47<00:00,  1.59s/it]


20230327_VD-N1-S-369.007-M-Loop_1112_1141.csv has been saved in /kaggle/working.
missing files = []
files not found in dataset = []
empty files = []
VDID not found in files = []




100%|██████████| 30/30 [00:47<00:00,  1.58s/it]


20230329_VD-N1-S-369.007-M-Loop_1112_1141.csv has been saved in /kaggle/working.
missing files = []
files not found in dataset = []
empty files = []
VDID not found in files = []


Email sent.
